In [1]:
import pandas as pd
import numpy as np
import torch
import torch.optim as optim
import warnings
warnings.filterwarnings('ignore')

input_dim = 5
hidden_dim = 6
output_dim = 1

EPOCHS = 400

X_data = []
X_test_data = []
Y_data = []

df_train = pd.read_csv("/kaggle/input/titanic/train.csv")   ## DataFrame은 엑셀표 의미
df_test = pd.read_csv("/kaggle/input/titanic/test.csv")

1. PassengerId: 승객의 고유 아이디
2. Survived: 생존 여부. 0은 사망, 1은 생존
3. Pclass: 티켓의 클래스. 1은 1등석, 2는 2등석, 3은 3등석
4. Name: 승객의 이름
5. Sex: 승객의 성별
6. Age: 승객의 나이
7. SibSp: 함께 탑승한 형제자매 또는 배우자의 수
8. Parch: 함께 탑승한 부모 또는 자녀의 수
9. Ticket: 티켓 번호
10. Fare: 승객이 지불한 요금
11. Cabin: 승객이 머무른 객실 번호
12. Embarked: 승객이 탑승한 항구. C는 Cherbourg, Q는 Queenstown, S는 Southampton을 의미.

## Data Preprocessing

### 결측치가 많은 age 변수의 특징을 Name 변수의 last name을 가져와서 분석 후 대입시킨다

In [2]:
df_train['last_name'] = df_train.Name.map( lambda x: x.split(',')[1].split( '.' )[0].strip())

# inspect the amount of people for each title
df_train['last_name'].value_counts()
display(df_train)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,last_name
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,Mr
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,Mrs
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,Miss
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,Mrs
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,Mr
...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S,Rev
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S,Miss
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S,Miss
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C,Mr


In [3]:
df_test['last_name'] = df_test.Name.map( lambda x: x.split(',')[1].split( '.' )[0].strip())

df_test['last_name'].value_counts()
display(df_test)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,last_name
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q,Mr
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S,Mrs
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q,Mr
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S,Mr
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S,Mrs
...,...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S,Mr
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C,Dona
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S,Mr
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S,Mr


In [4]:
df_train['last_name'] = df_train['last_name'].replace('Mlle', 'Miss')
df_train['last_name'] = df_train['last_name'].replace(['Mme','Lady','Ms'], 'Mrs')
df_train.last_name.loc[(df_train.last_name !=  'Master') & (df_train.last_name !=  'Mr') & (df_train.last_name !=  'Miss') 
             & (df_train.last_name !=  'Mrs')] = 'Others'

# inspect the amount of people for each title
df_train['last_name'].value_counts()

last_name
Mr        517
Miss      184
Mrs       128
Master     40
Others     22
Name: count, dtype: int64

In [5]:
df_test['last_name'] = df_test['last_name'].replace('Mlle', 'Miss')
df_test['last_name'] = df_test['last_name'].replace(['Mme','Lady','Ms'], 'Mrs')
df_test.last_name.loc[(df_test.last_name !=  'Master') & (df_test.last_name !=  'Mr') & (df_test.last_name !=  'Miss') 
             & (df_test.last_name !=  'Mrs')] = 'Others'

# inspect the amount of people for each title
df_test['last_name'].value_counts()

last_name
Mr        240
Miss       78
Mrs        73
Master     21
Others      6
Name: count, dtype: int64

In [6]:
df_train[['last_name','Age','SibSp']].groupby(['last_name']).mean()

,Age,SibSp
last_name,,
Master,4.574167,2.300000
Miss,21.804054,0.706522
Mr,32.368090,0.288201
Mrs,35.828829,0.687500
Others,45.428571,0.318182


In [7]:
df_train[['last_name','Age','SibSp']].groupby(['last_name']).std()

,Age,SibSp
last_name,,
Master,3.619872,1.910833
Miss,12.904198,1.426050
Mr,12.708793,0.821298
Mrs,11.416793,0.598555
Others,12.060561,0.646335


In [8]:
df_train[['SibSp','Age']].groupby(['SibSp']).mean()

,Age
SibSp,
0,31.397558
1,30.089727
2,22.620000
3,13.916667
4,7.055556
5,10.200000
8,NaN


In [9]:
df_train[['Pclass','Age']].groupby(['Pclass']).mean()

,Age
Pclass,
1,38.233441
2,29.877630
3,25.140620


In [10]:
df_train = pd.get_dummies(df_train, columns=['last_name'])
df_test = pd.get_dummies(df_test, columns=['last_name'])
display(df_train)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,last_name_Master,last_name_Miss,last_name_Mr,last_name_Mrs,last_name_Others
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,False,False,True,False,False
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,False,False,False,True,False
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,False,True,False,False,False
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,False,False,False,True,False
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S,False,False,False,False,True
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S,False,True,False,False,False
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S,False,True,False,False,False
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C,False,False,True,False,False


In [11]:
# Random Forest Regressor
from sklearn.ensemble import RandomForestRegressor


In [12]:
# notice that instead of using Title, we should use its corresponding dummy variables 
df_train_sub = df_train[['Age','last_name_Master','last_name_Miss','last_name_Mr','last_name_Mrs','last_name_Others','SibSp']]

X_train  = df_train_sub.dropna().drop('Age', axis=1)
y_train  = df_train['Age'].dropna()
X_test = df_train_sub.loc[np.isnan(df_train.Age)].drop('Age', axis=1)

regressor = RandomForestRegressor(n_estimators = 300) 
regressor.fit(X_train, y_train)
y_pred = np.round(regressor.predict(X_test),1)
df_train.Age.loc[df_train.Age.isnull()] = y_pred

df_train.Age.isnull().sum(axis=0) # no more NAN now

0

In [13]:
# notice that instead of using Title, we should use its corresponding dummy variables 
df_test_sub = df_test[['Age','last_name_Master','last_name_Miss','last_name_Mr','last_name_Mrs','last_name_Others','SibSp']]

X_train  = df_test_sub.dropna().drop('Age', axis=1)
y_train  = df_test['Age'].dropna()
X_test = df_test_sub.loc[np.isnan(df_test.Age)].drop('Age', axis=1)

regressor = RandomForestRegressor(n_estimators = 300)
regressor.fit(X_train, y_train)
y_pred = np.round(regressor.predict(X_test),1)
df_test.Age.loc[df_test.Age.isnull()] = y_pred

df_test.Age.isnull().sum(axis=0) # no more NAN now

0

### 나이의 임의의 구간 지정 

In [14]:
bins = [ 0, 4, 12, 18, 30, 50, 65, 100] # 임시 지정 
age_index = (1,2,3,4,5,6,7) #('baby','child','teenager','young','mid-age','over-50','senior')
df_train['Age-bin'] = pd.cut(df_train.Age, bins, labels=age_index).astype(int)

df_train[['Age-bin', 'Survived']].groupby(['Age-bin'],as_index=False).mean()

,Age-bin,Survived
0,1,0.675000
1,2,0.432432
2,3,0.394737
3,4,0.392027
4,5,0.345845
5,6,0.375000
6,7,0.125000


In [15]:
bins = [ 0, 4, 12, 18, 30, 50, 65, 100] # 임시 지정 
age_index = (1,2,3,4,5,6,7) #('baby','child','teenager','young','mid-age','over-50','senior')
df_test['Age-bin'] = pd.cut(df_test.Age, bins, labels=age_index).astype(int)

# df_test[['Age-bin', 'Survived']].groupby(['Age-bin'],as_index=False).mean()

In [16]:
df_train = df_train.drop(df_train.filter(like='last_name').columns, axis=1)
df_test = df_test.drop(df_test.filter(like='last_name').columns, axis=1)
display(df_train)
display(df_test)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Age-bin
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,4
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,5
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,4
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,5
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S,4
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S,4
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,20.1,1,2,W./C. 6607,23.4500,NaN,S,4
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C,4


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Age-bin
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q,5
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S,5
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q,6
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S,4
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S,4
...,...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,30.1,0,0,A.5. 3236,8.0500,NaN,S,5
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C,5
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S,5
416,1308,3,"Ware, Mr. Frederick",male,30.1,0,0,359309,8.0500,NaN,S,5


In [17]:
# Age 변수를 int형으로 변환
df_train['Age'] = df_train['Age'].fillna(0).astype(int)  # 결측치는 0으로 대체 후 int로 변환
df_test['Age'] = df_test['Age'].fillna(0).astype(int)

## Data Scaler

In [18]:
from sklearn.preprocessing import StandardScaler

# 훈련 데이터에 스케일러 적합
scaler = StandardScaler()

df_train[['Age']] = scaler.fit_transform(df_train[['Age']])

# 테스트 데이터에도 동일한 스케일러 사용
df_test[['Age']] = scaler.transform(df_test[['Age']])


In [19]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          891 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
 12  Age-bin      891 non-null    int64  
dtypes: float64(2), int64(6), object(5)
memory usage: 90.6+ KB


## Modeling

In [20]:
from torch.utils.data import DataLoader, TensorDataset


In [21]:
# 학습 데이터 준비
def data():
    global X_data, X_test_data, Y_data, dataloader, testloader
    
    Y = df_train['Survived']
    
    df_train.loc[df_train['Sex'] == 'male','Sex'] = 0
    df_train.loc[df_train['Sex'] == 'female','Sex'] = 1
    df_train['Sex'] = pd.to_numeric(df_train['Sex'])
    
    df_test.loc[df_test['Sex'] == 'male','Sex'] = 0
    df_test.loc[df_test['Sex'] == 'female','Sex'] = 1
    df_test['Sex'] = pd.to_numeric(df_test['Sex'])
    
    features = ['Pclass','Sex','SibSp','Parch','Age']   ## 유의미한 변수라고 생각해서 기본적으로 쓰는 변수. 수정 가능 / Age 추가
    X = pd.get_dummies(df_train[features], drop_first=True)
    X_test = pd.get_dummies(df_test[features], drop_first=True)

    
    X_data = torch.from_numpy(X.to_numpy()).float()
    X_test_data = torch.from_numpy(X_test.to_numpy()).float()  ## numpy로 변환하여 pytorch tensor로 전환
    Y_data = torch.from_numpy(Y.to_numpy()).float()
    
    # 데이터셋과 DataLoader 준비
    train_dataset = TensorDataset(X_data, Y_data)
    test_dataset = TensorDataset(X_test_data)
    
    # DataLoader로 배치 처리
    dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
    testloader = DataLoader(test_dataset, batch_size=64, shuffle=False)
    

In [22]:
class TitanicModel(torch.nn.Module):
    
    def __init__(self):
        super(TitanicModel, self).__init__()
        
        self.linear1 = torch.nn.Linear(input_dim, hidden_dim)
        self.relu  = torch.nn.ReLU()
        self.dropout = torch.nn.Dropout(0.2)  # 드롭아웃 추가
        self.linear3 = torch.nn.Linear(hidden_dim, output_dim)
        
    def forward(self, x):
        x = self.linear1(x)
        x = self.relu(x)  
        x = self.dropout(x)
        x = self.linear3(x)  # 출력 레이어
        return torch.sigmoid(x)
    
    
def train_one_epoch():
    global dataloader  # DataLoader를 사용
    
    running_loss = 0.0
    for inputs, labels in dataloader:
        labels = labels.float().view(-1, 1)  # 라벨 형태 맞추기

        optimizer.zero_grad()
        outputs = model(inputs)
        
        loss = loss_fn(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    
    avg_loss = running_loss / len(dataloader)  # 배치당 평균 손실 계산
    return avg_loss

In [23]:
# 전체 코드 실행 흐름
data()
model = TitanicModel()
loss_fn = torch.nn.BCELoss() 
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-8, weight_decay=0.001)

for epoch in range(EPOCHS):
    model.train(True)
    avg_loss = train_one_epoch()
    print(f"Epoch {epoch+1}/{EPOCHS}, Loss: {avg_loss}")
    
sum_correct = 0
total_num = len(X_data)

for i in range(total_num):
    result = model(X_data[i])
    result = result.detach().numpy()
    result = 1 if result > 0.5 else 0
    
    if result == Y_data[i]:
        sum_correct += 1
print('Accuracy : ', sum_correct/total_num)

submission_df = pd.DataFrame(columns = ['PassengerId', 'Survived'])
submission_df['PassengerId'] = df_test['PassengerId']

for i in range(len(X_test_data)):
    result = model(X_test_data[i])
    result = result.detach().numpy()
    result = 1 if result > 0.5 else 0
    submission_df.loc[i, 'Survived'] = result
    
submission_df.to_csv('submission.csv', header=True, index=False)
print('Result saved')

Epoch 1/400, Loss: 0.7130781199250903
Epoch 2/400, Loss: 0.7010674348899296
Epoch 3/400, Loss: 0.6877915220601218
Epoch 4/400, Loss: 0.6822697051933834
Epoch 5/400, Loss: 0.6735534582819257
Epoch 6/400, Loss: 0.6718386283942631
Epoch 7/400, Loss: 0.6613739856651851
Epoch 8/400, Loss: 0.6660600006580353
Epoch 9/400, Loss: 0.6572442991392953
Epoch 10/400, Loss: 0.6457949451037815
Epoch 11/400, Loss: 0.6462072006293705
Epoch 12/400, Loss: 0.6423021469797406
Epoch 13/400, Loss: 0.6403671034744808
Epoch 14/400, Loss: 0.6320983256612506
Epoch 15/400, Loss: 0.6268744426114219
Epoch 16/400, Loss: 0.6252858085291726
Epoch 17/400, Loss: 0.6271972187927791
Epoch 18/400, Loss: 0.6157297747475761
Epoch 19/400, Loss: 0.6116651317902974
Epoch 20/400, Loss: 0.5994219439370292
Epoch 21/400, Loss: 0.5942480862140656
Epoch 22/400, Loss: 0.5954087163720813
Epoch 23/400, Loss: 0.5980496662003654
Epoch 24/400, Loss: 0.5861423185893467
Epoch 25/400, Loss: 0.5928436730589185
Epoch 26/400, Loss: 0.570750321660